In [1]:
try:
    import inotify
except ImportError as e:
    !pip install inotify

In [2]:
from dask.distributed import Client
import subprocess as subp
import os
import dask
import inotify.adapters

In [3]:
from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()

In [4]:
c = Client(cluster)
c

Client Scheduler: tcp://127.0.0.1:39177 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 50.39 GB


In [5]:
from nccl_example import NCCL_World, unique_id
import random
from dask.distributed import wait

In [6]:
class Dask_MPI_Demo:
    
    def __init__(self, client):
        self.client = client
    
    @staticmethod
    def func_parse_host_port_(address):
        if '://' in address:
            address = address.rsplit('://', 1)[1]
        host, port = address.split(':')
        port = int(port)
        return host, port
    
    @staticmethod
    def func_init_(workerId, nWorkers, uniqueId):
        w = dask.distributed.get_worker()
        print("Hello World! from ip=%s worker=%s/%d" % \
              (w.address, w.name, nWorkers))
        a = NCCL_World(workerId, nWorkers)
        a.create_builder(uniqueId)

        return a
    
    @staticmethod
    def func_get_size_(world, r):
        return world.get_clique_size()
    
    @staticmethod
    def func_get_rank_(world, r):
        return world.get_rank()
    
    @staticmethod
    def func_get_device_(world, r):
        return world.get_device()

    @staticmethod
    def func_test_all_reduce_(world, r):
        return world.test_all_reduce()

        
    def get_workers_(self):
        return list(map(lambda x: Dask_MPI_Demo.func_parse_host_port_(x), self.client.has_what().keys()))
    
    def init(self, uniqueId):
        workers = self.get_workers_()
        workers_indices = list(zip(workers, range(len(workers))))

        self.clique = [(idx, worker, self.client.submit(Dask_MPI_Demo.func_init_, 
                                           idx, 
                                           len(workers), 
                                           uniqueId,
                                           workers=[worker]))
             for worker, idx in workers_indices]
        
    def get_clique_size(self):
        return [c.submit(Dask_MPI_Demo.func_get_size_, a, random.random()).result() for i, w, a in self.clique]

    def get_device(self):
        return [c.submit(Dask_MPI_Demo.func_get_device_, a, random.random()).result() for i, w, a in self.clique]

    def get_rank(self):
        return [c.submit(Dask_MPI_Demo.func_get_rank_, a, random.random()).result() for i, w, a in self.clique]
    
    def test_all_reduce(self):
        f = [c.submit(Dask_MPI_Demo.func_test_all_reduce_, a, random.random()) for i, w, a in self.clique])
        wait(f)
        return [a.result() for a in f]
        


In [7]:
local_world = unique_id()
local_world

b'\x02\x00\x81\x0c\xc0\xa8\x01\xcf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00z\x7f\x00\x00\xc0w\xe6\xcb\xdaU\x00\x00\x10G\x92\xcd\xdaU\x00\x00>\xa2\xe5\x8bz\x7f\x00\x00\x00{\x97\xc9sS\x86X\xa0\x9e\x08\xce\xdaU\x00\x00\xcd\xbf\xba\x12\xcc\xb6\xf7\xc8\x04\x00\x00\x00\x00\x00\x00\x00\x7f\x00\x00\x00\x00\x00\x00\x00\x00\x0e\x1cp\xfd\x7f\x00\x000\xcd\xc6\xcb\xdaU\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\xa0,\x18\\z\x7f\x00'

In [8]:
demo = Dask_MPI_Demo(c)
demo.init(local_world)

In [9]:
demo.clique

[(0,
  ('127.0.0.1', 39159),
  <Future: status: pending, key: func_init_-3349a171c8ac5d33369e568e00ea79a9>),
 (1,
  ('127.0.0.1', 44513),
  <Future: status: pending, key: func_init_-4caaebc3ac15faee2b33ccf0c7c39b99>)]

In [10]:
demo.get_clique_size()

[2, 2]

In [11]:
demo.get_device()

[0, 0]

In [12]:
demo.get_rank()

[0, 1]

In [19]:
a = demo.test_all_reduce()

In [20]:
a

[<Future: status: finished, type: NoneType, key: func_test_all_reduce_-d5b2d985a667c02dc2fa2e5cf5526dfa>,
 <Future: status: finished, type: NoneType, key: func_test_all_reduce_-f9a2d53e49f62778268697d212a197a9>]

In [21]:
print(str(a[0].result()))

None
